In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('../input/titanic/train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#Let's study the dataset to begin doing the predictions
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
train['Ticket'].value_counts()

347082      7
CA. 2343    7
1601        7
3101295     6
CA 2144     6
           ..
9234        1
19988       1
2693        1
PC 17612    1
370376      1
Name: Ticket, Length: 681, dtype: int64

In [5]:
#those people have the same ticket number, therefore they bought together and probably where traveling together.
group_tickets = train[train['Ticket'].map(train['Ticket'].value_counts() > 1)]
print(group_tickets.shape)

(344, 12)


In [6]:
#There are too many missing values in cabin, they are cabinless people? This was a mistake? Probably, should drop this column.
print('No cabin = %s' %train.Cabin.isna().sum())

No cabin = 687


In [7]:
train.drop(columns=('Cabin'), inplace=True)
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [8]:
#Large number os na, but is possible to deal with it.
print('No age = %s' %train.Age.isna().sum())

No age = 177


In [9]:
#I have a choice here, I could try to fill in the nan values in age, either with median, mean or
#imputing values from other catgories to take an avarage mean. Or drop the nans.
#I think the number still will be large enough to train. So I'll go with the drop option.
train.dropna(subset=['Age'], inplace=True)
print('No age = %s' %train.Age.isna().sum())

No age = 0


In [10]:
#Do the place you emkared affects your survival chance? I mean... It shouldn't, but maybe it could affect where
#you will be in the ship.
train['Embarked'].value_counts()

S    554
C    130
Q     28
Name: Embarked, dtype: int64

In [11]:
train.replace('S', 0, inplace=True)
train.replace('C', 1, inplace=True)
train.replace('Q', 2, inplace=True)
train['Embarked'].value_counts()

0.0    554
1.0    130
2.0     28
Name: Embarked, dtype: int64

In [12]:
train['Embarked'].isna().sum()

2

In [13]:
#Let's remove those nan
train.dropna(subset=['Embarked'], inplace =True)
train['Embarked'].isna().sum()

0

In [14]:
train['SibSp'].value_counts()

0    469
1    183
2     25
4     18
3     12
5      5
Name: SibSp, dtype: int64

In [15]:
train['Parch'].value_counts()

0    519
1    110
2     68
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

In [16]:
#Let's convert the gender variable in numbers
train.replace('male', 1, inplace=True)
train.replace('female', 0, inplace=True)
train['Sex'].value_counts()

1    453
0    259
Name: Sex, dtype: int64

In [17]:
train.replace('male', 1, inplace=True)
train.replace('female', 0, inplace=True)

In [18]:
#Selecting the variables
X = train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = train['Survived']

In [19]:
print(X.describe())
print(X.isna().sum())

           Pclass         Sex         Age       SibSp       Parch        Fare  \
count  712.000000  712.000000  712.000000  712.000000  712.000000  712.000000   
mean     2.240169    0.636236   29.642093    0.514045    0.432584   34.567251   
std      0.836854    0.481420   14.492933    0.930692    0.854181   52.938648   
min      1.000000    0.000000    0.420000    0.000000    0.000000    0.000000   
25%      1.000000    0.000000   20.000000    0.000000    0.000000    8.050000   
50%      2.000000    1.000000   28.000000    0.000000    0.000000   15.645850   
75%      3.000000    1.000000   38.000000    1.000000    1.000000   33.000000   
max      3.000000    1.000000   80.000000    5.000000    6.000000  512.329200   

         Embarked  
count  712.000000  
mean     0.261236  
std      0.521561  
min      0.000000  
25%      0.000000  
50%      0.000000  
75%      0.000000  
max      2.000000  
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embark

In [20]:
test = pd.read_csv('../input/titanic/test.csv')
test.replace('male', 1, inplace=True)
test.replace('female', 0, inplace=True)
test.replace('S', 0, inplace=True)
test.replace('C', 1, inplace=True)
test.replace('Q', 2, inplace=True)

In [21]:
X_test = test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
X_test.replace('male', 1, inplace=True)
X_test.replace('female', 0, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [22]:
#Well... now we have to deal with it...
print(X_test.describe())
print(X_test.isna().sum())
print(X_test.shape)
print(test.shape)

           Pclass         Sex         Age       SibSp       Parch        Fare  \
count  418.000000  418.000000  332.000000  418.000000  418.000000  417.000000   
mean     2.265550    0.636364   30.272590    0.447368    0.392344   35.627188   
std      0.841838    0.481622   14.181209    0.896760    0.981429   55.907576   
min      1.000000    0.000000    0.170000    0.000000    0.000000    0.000000   
25%      1.000000    0.000000   21.000000    0.000000    0.000000    7.895800   
50%      3.000000    1.000000   27.000000    0.000000    0.000000   14.454200   
75%      3.000000    1.000000   39.000000    1.000000    0.000000   31.500000   
max      3.000000    1.000000   76.000000    8.000000    9.000000  512.329200   

         Embarked  
count  418.000000  
mean     0.464115  
std      0.685516  
min      0.000000  
25%      0.000000  
50%      0.000000  
75%      1.000000  
max      2.000000  
Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1


In [23]:
#I need to deal with those na, I will start with Fare, that is only one and use the mean.
X_test['Fare'] = X_test['Fare'].fillna(X_test['Fare'].mean())
X_test['Fare'].isna().sum()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0

In [24]:
#Let's try to use knnimputer to solve the nan at age.
from sklearn.impute import KNNImputer
Before_age = X_test[['Age']]
imputer = KNNImputer(n_neighbors=2)
After_age = imputer.fit_transform(Before_age)
X_test['Age'] = After_age
X_test['Age'].isna().sum()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0

In [25]:
#Lets model
from sklearn.tree import DecisionTreeRegressor
import numpy as np


SEED = 1405
np.random.seed(SEED)
model = DecisionTreeRegressor()
model.fit(X,y)
Predictions = model.predict(X_test)

In [26]:
test['Survived'] = Predictions
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,2,0.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,0,0.0
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,2,1.0
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,0,1.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,0,0.0


In [27]:
submission = test[['PassengerId', 'Survived']]
submission['Survived'] = submission['Survived'].astype(int)
print(submission.shape)
submission.head()

(418, 2)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,0


In [28]:
submission.to_csv('./submission.csv', index=False)